# Training e prime valutazioni

In [14]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans,DBSCAN
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [15]:
import os
import sys
#SERVE PER POTER IMPORTARE I FILE .PY DALLA CARTELLA SOURCE
sys.path.append('..')
sys.path.append('..\data')

# importo i miei script dal package src
from src.data_load_notebook  import *
#import pandas as pd|
# ||||||||||è un utilizzo un pò improprio di import, basta ricordare che così importiamo pandas
from src.classes import Spettri,plot_peaks, plot_spettri
from src import funzioni
from src import cluster_routine
import numpy as np

from scipy.spatial.distance import pdist
from numpy.random import choice
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pass

In [16]:
#routine per estrarre i picchi e le feature dai csampioni
pk1 = Spettri(data1bkg)
pk1bkg = Spettri(data1)
pk1.peakfinder()
pk1bkg.peakfinder()
pk1.featextract()
pk1bkg.featextract()
pk1.featextract2()
pk1bkg.featextract2()
pass

In [17]:
picchidf = funzioni.peakfinder(database)
database_feat1 = funzioni.featextract1_df(picchidf)
database_feat2 = funzioni.featextract2_df(picchidf)

In [18]:
# creo due pipeline per i due spazi
steps= [('scaler',StandardScaler()),('pca',PCA()),('clustering',KMeans())]
param_g = {'pca__n_components': [2,4,6],'clustering__n_clusters': [3,5,6,7]}
pipekm = Pipeline(steps)
search = GridSearchCV(pipekm, param_grid= param_g )
search.fit(pk1.feature)
#dtbs_feat1_trnsf = search.transform()

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('clustering', KMeans())]),
             param_grid={'clustering__n_clusters': [3, 5, 6, 7],
                         'pca__n_components': [2, 4, 6]})

In [ ]:
feat1transf = pd.DataFrame(search.best_estimator_[:-1].transform(pk1.feature))
feat1transf['labels']= search.best_estimator_.predict(pk1.feature)
plt.figure(figsize = (17,8))
sns.scatterplot(data = feat1transf,x = 0, y = 1, hue= 'labels')

In [ ]:
lab = search.best_estimator_.predict(database_feat1)
databasetranform = pd.DataFrame(search.best_estimator_[:-1].fit_transform(database_feat1))
databasetranform['labels'] = lab
plt.figure(figsize = (17,8))
sns.scatterplot(data = databasetranform, x = 0, y = 1, hue = 'labels')

In [25]:
dfpk1 = pd.DataFrame(pk1.feature, copy=True)
dfpk1['labels'] = search.predict(pk1.feature)
clust_dist_mean = []
for x in np.unique(dfpk1['labels']):
    clust_dist_mean.append(np.mean(pdist(dfpk1[dfpk1['labels']== x ].drop('labels',axis = 1) ) ))

In [20]:
np.mean(pdist(dfpk1.drop('labels',axis = 1)))

292.46291367869514

In [27]:
np.mean(clust_dist_mean)

190.82670975461647

In [29]:
search.cv_results_

{'mean_fit_time': array([0.03473802, 0.03922133, 0.03626447, 0.04996347, 0.03970761,
        0.03492756, 0.04781623, 0.04687209, 0.04687176, 0.04946423,
        0.04981585, 0.05174332]),
 'std_fit_time': array([0.00571119, 0.00663573, 0.00644596, 0.01520397, 0.00710625,
        0.00606849, 0.00188746, 0.00988129, 0.00988174, 0.00748089,
        0.00588729, 0.00635148]),
 'mean_score_time': array([0.07518225, 0.08054881, 0.08157911, 0.07521358, 0.07803354,
        0.08078628, 0.07767329, 0.07882204, 0.07624102, 0.07832785,
        0.07567592, 0.07667422]),
 'std_score_time': array([0.00586572, 0.00454483, 0.00752018, 0.00628363, 0.00017183,
        0.00533423, 0.00089221, 0.00140529, 0.00708723, 0.00041702,
        0.00488708, 0.00289059]),
 'param_clustering__n_clusters': masked_array(data=[3, 3, 3, 5, 5, 5, 6, 6, 6, 7, 7, 7],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
            